In [1]:
from functools import wraps


def func_logger(fn):
    @wraps(fn)
    def inner(*args, **kwargs):
        result = fn(*args, **kwargs)
        print(f"LOG: {fn.__qualname__}({args}{kwargs}) = {result}")
        return result
    return inner


def class_logger(cls):
    for name, obj in vars(cls).items():
        if callable(obj):
            print("decorating", cls, name)
            setattr(cls, name, func_logger(obj))
    return cls


In [2]:
@class_logger
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age

    def greet(self):
        return f"Hi! My name is {self.name} and I'm {self.age} years old."

decorating <class '__main__.Person'> __init__
decorating <class '__main__.Person'> greet


In [3]:
Person("Bob", 10).greet()

LOG: Person.__init__((<__main__.Person object at 0x105301ee0>, 'Bob', 10){}) = None
LOG: Person.greet((<__main__.Person object at 0x105301ee0>,){}) = Hi! My name is Bob and I'm 10 years old.


"Hi! My name is Bob and I'm 10 years old."

In [4]:
# let's do the same with Metaclasses
class ClassLogger(type):
    def __new__(mcls, name, bases, class_dict):
        cls = super().__new__(mcls, name, bases, class_dict)
        for name, obj in cls.__dict__.items():
            if callable(obj):
                print("decorating", cls, name)
                setattr(cls, name, func_logger(obj))
        return cls

In [5]:
class Person(metaclass=ClassLogger):
    def __init__(self, name, age):
        self.name = name
        self.age = age

    def greet(self):
        return f"Hi! My name is {self.name} and I'm {self.age} years old."

decorating <class '__main__.Person'> __init__
decorating <class '__main__.Person'> greet


In [6]:
p = Person("Bob", 34)

LOG: Person.__init__((<__main__.Person object at 0x10532cb00>, 'Bob', 34){}) = None


In [7]:
p.greet()

LOG: Person.greet((<__main__.Person object at 0x10532cb00>,){}) = Hi! My name is Bob and I'm 34 years old.


"Hi! My name is Bob and I'm 34 years old."

In [8]:
@class_logger
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age
    
    def greet(self):
        return f"Hi! My name is {self.name} and I'm {self.age} years old."


class Student(Person):
    def __init__(self, name, age, student_number):
        super().__init__(name, age)
        self.student_number = student_number

    def study(self):
        # new method that won't be decorated
        return f"{self.name} studies!"


decorating <class '__main__.Person'> __init__
decorating <class '__main__.Person'> greet


In [9]:
s = Student("Alex", 22, "A12345")

LOG: Person.__init__((<__main__.Student object at 0x10532dd30>, 'Alex', 22){}) = None


In [10]:
s.greet()

LOG: Person.greet((<__main__.Student object at 0x10532dd30>,){}) = Hi! My name is Alex and I'm 22 years old.


"Hi! My name is Alex and I'm 22 years old."

In [11]:
s.study()  # no decorator here

'Alex studies!'

In [12]:
class Student(Person):
    def __init__(self, name, age, student_number):
        super().__init__(name, age)
        self.student_number = student_number

    def greet(self):
        # overwrites old `greet` - is not decorated
        return f"{self.name} sends greetings!"


In [13]:
s = Student("Bob", 52, "S2132")
s.greet()

LOG: Person.__init__((<__main__.Student object at 0x10532db50>, 'Bob', 52){}) = None


'Bob sends greetings!'

In [14]:
class Person(metaclass=ClassLogger):
    def __init__(self, name, age):
        self.name = name
        self.age = age

    def greet(self):
        return f"Hi! My name is {self.name} and I'm {self.age} years old."


# metaclass is being inherited all the way down
class Student(Person):
    def __init__(self, name, age, student_number):
        super().__init__(name, age)
        self.student_number = student_number

    def greet(self):
        return f"{self.name} sends greetings!"


decorating <class '__main__.Person'> __init__
decorating <class '__main__.Person'> greet
decorating <class '__main__.Student'> __init__
decorating <class '__main__.Student'> greet


In [15]:
# metaclasses 
s = Student("John", 23, "C9321")
s.greet()

LOG: Person.__init__((<__main__.Student object at 0x10532f020>, 'John', 23){}) = None
LOG: Student.__init__((<__main__.Student object at 0x10532f020>, 'John', 23, 'C9321'){}) = None
LOG: Student.greet((<__main__.Student object at 0x10532f020>,){}) = John sends greetings!


'John sends greetings!'